# Train Models Using LeRobot on MI300x

This guide walks you through setting up environment for training imitation learning policies using LeRobot library on a DigitalOcean (DO) instance equipped with AMD MI300x GPUs and ROCm.

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=${HF_USER}/${DATASET_NAME}`).
 If you don’t have an access token yet, you can sign up for Hugging Face [here](https://huggingface.co/join). After signing up, create an access token by visiting [here](https://huggingface.co/settings/tokens).
- A wandb account to enable training visualization and upload your training evidence to our github.
 You can sign up for Wandb [here](https://wandb.ai/signup) and visit [here](https://wandb.ai/authorize) to create a token.
- Access to DO instance AMD Mi300x GPU


## Verify ROCm and GPU availability
This cell uses `pytorch` to check AMD GPU Info. The expected ouput is 
`
CUDA compatible device availability: True
device name [0]: AMD Instinct MI300X VF
`

In [ ]:
import torch
print(f'CUDA compatible device availability:',torch.cuda.is_available())
print(f'device name [0]:', torch.cuda.get_device_name(0))


## Install FFmpeg 7.x
This cell uses `apt` to install ffmpeg 7.x for LeRobot.

In [ ]:
!add-apt-repository ppa:ubuntuhandbook1/ffmpeg7 -y # install PPA which contains ffmpeg 7.x
!apt update && apt install ffmpeg -y

## Install LeRobot v0.4.1
This cell clones the `lerobot` repository from Hugging Face, and installs the package in editable mode. Extra Features: To install additional dependencies for training SmolVLA or Pi models, refer to the [LeRobot offical page](https://huggingface.co/docs/lerobot/index). 


In [ ]:
!git clone https://github.com/huggingface/lerobot.git
!cd lerobot && git checkout -b v0.4.1 v0.4.1 # let’s synchronize using this version
!cd lerobot && pip install -e .

## Weights & Biases login
This cell install and log into Weights & Biases (wandb) to enable experiment tracking and logging.

In [ ]:
!pip install wandb
import wandb
wandb.login(key="your_wandb_token")

## Login into Hugging Face Hub

In [ ]:
from huggingface_hub import login
login(token="your_huggingface_token")

## Start Training Models with LeRobot

This cell uses the lerobot-train CLI from the lerobot library to train a robot control policy. 

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`: 
 Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `lerobot/svla_so100_pickplace`.

2. `--policy.type=act`: 
 Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`: 
 Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`: 
 A name for this training job, used for logging and Weights & Biases.The name typically includes the model type (e.g., act, smolvla), the dataset name, and additional descriptive tags.

5. `--policy.device=cuda`: 
 Use `cuda` if training on an AMD or NVIDIA GPU. 

6. `--wandb.enable=true`: 
 Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

7. `--policy.push_to_hub=`:

 Enables automatic uploading of the trained policy to the Hugging Face Hub. You must specify `--policy.repo_id` (e.g., ${HF_USER}/{REPO_NAME}) if it is True.

In [ ]:
!lerobot-train \
 --dataset.repo_id=ichbinblau/so101_stack2cubes_dataset \
 --batch_size=64 \
 --steps=1000 \
 --output_dir=outputs/train/act_so101_3cube_1ksteps \
 --job_name=act_so101_3cube_1ksteps \
 --policy.repo_id=ichbinblau/so101_act_stack2cubes \
 --policy.device=cuda \
 --policy.type=act \
 --policy.push_to_hub=true \
 --wandb.enable=true

**Notes**:

- If using a local dataset, add `--dataset.root=/path/to/dataset`.
- Adjust `--batch_size` and `--steps` based on your hardware and dataset.
- Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`
- Training progress visualized in your wandb dashboard


## Download Models from Hugging Face to Local Machine
Now after training is done, download the model to local machine. 

In [ ]:
!huggingface-cli download ${HF_USER}/{REPO_NAME} --repo-type model --local-dir path/to/model
# e.g. huggingface-cli upload ${HF_USER}/act_so101_3cube_1ksteps \
# outputs/train/act_so101_3cube_1ksteps/checkpoints/last/pretrained_model

## Miscs
1. Once the environment is setup, you can open a terminal session for training by navigating to `File → New Launcher → Other → Terminal`.
2. You can also upload your datasets to the container by clicking the `Upload Files` button in the left pane.

## Q&A
1. If you encounter an error like:
 ```
 FileExistsError: Output directory outputs/train/act_so101_3cube_1ksteps already exists and resume is False. Please change your output directory so that outputs/train/act_so101_3cube_1ksteps is not overwritten. 
 ```
 Remove the existing directory before proceeding:

In [ ]:
!rm -fr outputs/train/act_so101_3cube_1ksteps

2. When running models other than ACT, ensure you install the required additional dependencies for those models.

In [ ]:
# For smolVLA
!cd lerobot && pip install -e ".[smolvla]"
# For Pi
!cd lerobot && pip install -e ".[pi]"

3. If you want to resume the training from last checkpoint, run the command below:

In [ ]:
!lerobot-train \
 --resume=true \
 --config_path=outputs/train/<job name>/checkpoints/last/pretrained_model/train_config.json \
 --steps=<new total steps>

4. If you want to upload your dataset using `huggingface-cli upload <repo name> <path to the dataset> --repo-type=dataset`, be sure to set a codebase tag like below:

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import login

login(token="your_huggingface_token")
hub_api = HfApi()
hub_api.create_tag(<HF_REPO_NAME>, tag="v3.0", revision="main", repo_type="dataset")